In [ ]:
import os
import pyxdf # xdf file importation
import numpy as np
from my_functions import *

### Ensure that the root directory is the project directory
In some cases, the root directory may not be the project directory. This is usually the case when the notebook is run from a subdirectory of the project directory.

In [ ]:
# if pwd is SRC  change to root
if os.path.basename(os.getcwd()) == "SRC":
    os.chdir("..")
print("Current working directory: ", os.getcwd())



## Initialization

### File selection

In [ ]:
# Define the xdf file path
# FILENAME="020_DesMar_20211129_1_c.xdf"
# path=os.path.normpath("../DAT/Input/001_MolLud_20201112_1_c.xdf")
#input_filepath = os.path.normpath("DAT/INPUT/"+FILENAME)
input_filepath = xdf_filepath
INPUT_FILENAME = os.path.splitext(os.path.basename(input_filepath))[0]

print("Input filepath: ",input_filepath)
print("Input filename: ",INPUT_FILENAME)

### Loading streams of interest

#### Storing the data of interest in two variables (markers and signal) 

In [ ]:
# load_xdf returns selected streams as list of streams (ie.dictionary)
EEG_Stream, EEG_fileheader = pyxdf.load_xdf(input_filepath, select_streams=[{'type': 'EEG'}])
Mouse_markers_Stream, Mouse_markers_header = pyxdf.load_xdf(input_filepath, select_streams=[{'type': 'Markers', 'name': 'MouseToNIC'}])

if len(EEG_Stream) and len(Mouse_markers_Stream) ==1:
    EEG_Stream=EEG_Stream[-1] #acess to the only EEG stream of the list
    Mouse_markers_Stream=Mouse_markers_Stream[-1]
else:
    print("Multiple streams matching type restriction")


#### Defining the channels names

In [ ]:
# definition of the EEG channels' names
channels_dict = {"Channel_1": "C4",
                "Channel_2": "FC2",
                "Channel_3": "FC6",
                "Channel_4": "CP2",
                "Channel_5": "C3",
                "Channel_6": "FC1",
                "Channel_7": "FC5",
                "Channel_8": "CP1"}

### Tidy recoding parameters' data

In [ ]:
#get sampling rate
Srate = EEG_Stream["info"]["effective_srate"]

# Markers
# select the marker labels
Markers_labels = Mouse_markers_Stream["time_series"]

# Markers start with a time relative to the execution time of the recording
Marker_times = (
    Mouse_markers_Stream["time_stamps"]-EEG_Stream["time_stamps"][0])

# Creation of a 2D array Markers_times_labels: [[markers_timesstamps],[markers_labels]]
Markers_times_labels = np.column_stack((Marker_times, Markers_labels))

In [ ]:
# EEG
# format eeg times_stamps in seconds relative to the execution time of the recording
EEG_times = EEG_Stream["time_stamps"]-EEG_Stream["time_stamps"][0]
# get amplitudes recorded by each electrode of the recording set-up
EEG_raw_amplitudes = EEG_Stream["time_series"]

### Export

#### Data to be exported

In [ ]:
print("PROCESS_SIGNAL --",PROCESS_SIGNAL)

In [ ]:
if PROCESS_SIGNAL is True:
    print("Detrending -----------------")
    EEG_amplitudes_centered=detrend_signals(EEG_raw_amplitudes)
    print("Rereferencing --------------")
    EEG_amplitudes_rereferenced=rereference_signals(input_signals=EEG_amplitudes_centered)
    print("Filtering ------------------")
    EEG_amplitudes_centered_filtered,_,_=filter_signal(input_signals=EEG_amplitudes_rereferenced,
                                                sample_rate=Srate,
                                                order=8,cutofffreq=(5,100,50))
    EEG_for_export=EEG_amplitudes_centered_filtered
    DATA_STATUS="preprocessed"
elif PROCESS_SIGNAL is False:
    print("Raw signal...")
    EEG_for_export=EEG_raw_amplitudes
    DATA_STATUS="raw"
else:
    raise ValueError('PROCESS_SIGNAL must be Boolean (True or False)')


print(f"EEG_for_export shape : {EEG_for_export.shape}")

In [ ]:
#Stack columns as electrodes signals followed by column of timestamps
times_amplitudes=np.column_stack((EEG_for_export,EEG_times))

#### Export to CSV file

In [ ]:
#Create header for CSV
header = ', '.join([f"{key}:{value}" for key, value in channels_dict.items()])
header=header+',time(sec)'
print("export data header :",header)


In [ ]:
#Create filename
#originalfilename=INPUT_FILENAME.replace(".xdf","")

exportfilename=f"{INPUT_FILENAME}_{DATA_STATUS}_{round(Srate,3)}_Hz.csv"

exportfilepath=os.path.normpath("DAT/INPUT/"+exportfilename)
print(f"Input filepath : {input_filepath}")
print(f"Output filename : {exportfilename}")
print(f"Output filepath : {exportfilepath}")

In [ ]:
#np.savetxt(exportfilepath,times_amplitudes,delimiter=",")
np.savetxt(exportfilepath, times_amplitudes, delimiter=',', header=header, comments='', fmt='%d')